In [1]:
import numpy as np
import string
import time
import datetime
import os

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import emotion_model
import dwdii_transforms

Using Theano backend.


In [2]:
imagePath = "/root/facial_expressions/images"
dataPath = "/root/facial_expressions/data/legend.csv"

In [3]:
os.listdir('/root/facial_expressions/data')

['500_picts_satz.csv', 'legend.csv']

### Load Training and Test Data

In [4]:
emoMetaData = dwdii_transforms.load_training_metadata(dataPath)

In [5]:
len(emoMetaData)

13682

In [6]:
maxData = 2000
X_data, Y_data = dwdii_transforms.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 200)
print X_data.shape
print Y_data.shape

0.000000: King_Abdullah_II_0001.jpg
0.100000: George_W_Bush_0193.jpg
0.200000: Jason_Alexander_0002.jpg
0.300000: Tammy_Lynn_Michaels_0002.jpg
0.400000: Deece_Eckstein_0001.jpg
0.500000: Carrie-Anne_Moss_0002.jpg
0.600000: George_W_Bush_0494.jpg
0.700000: Jim_Furyk_0001.jpg
0.800000: HrithikRoshan_72.jpg
0.800000: Debra_Brown_0002.jpg
0.900000: Emmanuelle_Jagodsinski_0001.jpg
(2000, 350, 350)
(2000, 1)


The following code segment splits the data into training and test data sets. Currently this is just a standard 80/20 split for training and test respectively. A random.shuffle should be probably be added here as well.

In [7]:
# Split the data into Training and Test sets
trainNdx = int(maxData * .8)
print trainNdx
X_train, X_test = np.split(X_data, [trainNdx])
Y_train, Y_test = np.split(Y_data, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

1600
(1600, 350, 350)
(400, 350, 350)
(1600, 1)
(400, 1)


### Transformations
In this section, we will apply transformations to the existing images to increase of training data, as well as add a bit of noise in the hopes of improving the overall training activities.

### Define the Model
In this section, we define the model. The `emotion_model` module contains the model definition itself. `emotion_model_v1` is a basic convolutional neural network.

In [8]:
# Map the emotions to integers for categorization later.
emotions = dwdii_transforms.emotionNumerics()
print emotions
print len(emotions)

{'sadness': 6, 'neutral': 2, 'disgust': 1, 'anger': 0, 'surprise': 4, 'fear': 5, 'happiness': 3}
7


In [9]:
model = emotion_model.emotion_model_v5(len(emotions), verbose=True)

Compiling Model ... 
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)    (None, 1, 352, 352) 0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)    (None, 32, 333, 333)12832       zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 166, 166)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)    (None, 32, 168, 168)0           maxpooling2d_1[0][0]             
______________________________________________________________________

### Training the Model

The following code segment trains the model using the `run_network` helper function. I seem to be hitting a memory issue (my interpretation), when I have batches above a certain threshold. Batches=10 work fine, but batches of 100 are too big. May need to allocate more RAM to the docker container.

In [ ]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, 350, 350)
trainX = X_train.reshape(X_train.shape[0], 1, 350, 350)

In [ ]:
print "Training start: " + str(datetime.datetime.now())
m, h = emotion_model.run_network([trainX, testX, Y_train, Y_test], model, batch=32, epochs=1)

Training start: 2016-11-16 01:01:39.589321
(1600, 7)
(400, 7)
Training model...
Train on 1600 samples, validate on 400 samples
Epoch 1/1


* Model v2: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 4474s 
  * #2: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 5469s 
* Model v3: Network's test score [loss, accuracy]: [8.0187525367736825, 0.50249999999999995] - 2499s
* Model v4.1: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 2137s 
* Movel v4.2: Network's test score [loss, accuracy]: [8.0187525367736825, 0.50249999999999995] - 3108s 

In [ ]:
model.save_weights("dwdii-emo-01v5.hdf5", overwrite=True)

### References

* OpenCV/CV2: http://askubuntu.com/questions/447409/how-to-install-opencv-2-9-for-python
* Docker Commit: http://stackoverflow.com/questions/19585028/i-lose-my-data-when-the-container-exits
  * docker ps -l
  * docker commit <ContainerID> dittenhafer/dl